In [1]:
# Import Splinter and BeautifulSoup
from splinter import Browser
from bs4 import BeautifulSoup as soup
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

In [2]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

# Scraping Text From redplanetscience.com

In [ ]:
# Visit the mars nasa news site
url = 'https://redplanetscience.com'
browser.visit(url)
# Optional delay for loading the page
browser.is_element_present_by_css('div.list_text', wait_time=1)

In [ ]:
html = browser.html
news_soup = soup(html, 'html.parser')
slide_elem = news_soup.select_one('div.list_text')

In [ ]:
slide_elem.find('div', class_='content_title')

In [ ]:
# Use the parent element to find the first `a` tag and save it as `news_title`
news_title = slide_elem.find('div', class_='content_title').get_text()
news_title

In [ ]:
# Use the parent element to find the paragraph text
news_p = slide_elem.find('div', class_='article_teaser_body').get_text()
news_p

# Scraping Images From spaceimages-mars.com

In [ ]:
# Visit URL
url = 'https://spaceimages-mars.com'
browser.visit(url)

In [ ]:
# Find and click the full image button
full_image_elem = browser.find_by_tag('button')[1]
full_image_elem.click()

In [ ]:
# Parse the resulting html with soup
html = browser.html
img_soup = soup(html, 'html.parser')

In [ ]:
# Find the relative image url
img_url_rel = img_soup.find('img', class_='fancybox-image').get('src')
img_url_rel

In [ ]:
# Use the base URL to create an absolute URL
img_url = f'https://spaceimages-mars.com/{img_url_rel}'
img_url

# Scraping From galaxyfacts-mars.com

In [ ]:
df = pd.read_html('https://galaxyfacts-mars.com')[0]
df.columns=['description', 'Mars', 'Earth']
df.set_index('description', inplace=True)
df

In [ ]:
df.to_html()

## D1: Scrape High-Resolution Mars' Hemisphere Images and Titles

In [3]:
# 1. Use browser to visit the URL 
url = 'https://marshemispheres.com/'

browser.visit(url)
html = browser.html
hemi_soup = soup(html, 'html.parser')

In [42]:
# 2. Create a list to hold the images and titles.
hemisphere_image_urls = []

links = hemi_soup.find_all('h3')
link = links[0].text

# 3. Write code to retrieve the image urls and titles for each hemisphere.
for link in links[0:-1]:
    try:
        # move to the next page
        browser.links.find_by_partial_text(link.text).click()
        
        # parse html
        img_html = browser.html
        img_soup = soup(img_html, 'html.parser')
        
        # extract partial image url and page title
        img_partial_url = img_soup.find('li').a.get('href')
        img_full_url = "https://marshemispheres.com/" + img_partial_url
        title = img_soup.find('h2', class_='title').text
        
        # add title and url to dictionary
        hemisphere_image_urls.append({'img_url':img_full_url, 'title':title})
        browser.back()
    except:
        print('exception')
        browser.back()
        

In [43]:
# 4. Print the list that holds the dictionary of each image url and title.
hemisphere_image_urls

[{'img_url': 'https://marshemispheres.com/images/full.jpg',
  'title': 'Cerberus Hemisphere Enhanced'},
 {'img_url': 'https://marshemispheres.com/images/schiaparelli_enhanced-full.jpg',
  'title': 'Schiaparelli Hemisphere Enhanced'},
 {'img_url': 'https://marshemispheres.com/images/syrtis_major_enhanced-full.jpg',
  'title': 'Syrtis Major Hemisphere Enhanced'},
 {'img_url': 'https://marshemispheres.com/images/valles_marineris_enhanced-full.jpg',
  'title': 'Valles Marineris Hemisphere Enhanced'}]

In [ ]:
# 5. Quit the browser
browser.quit()